# ETL & EDA

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

In [2]:
df = pd.read_csv('/Users/aminrimdans/Documents/GitHub/Naveena_Daniel_Data606/Exported_tables/recalls_raw_data.csv', low_memory=False)
#Github link to data above is https://github.com/NaveenaChodayy/TEAM_E_Data606/blob/main/Exported_tables/recalls_raw_data.csv

In [3]:
#checking dataframe shape, column names, and dtypes
df.shape,\
df.dtypes

((78184, 17),
 FEI Number                    object
 Recalling Firm Name           object
 Product Type                  object
 Product Classification        object
 Status                        object
 Distribution Pattern          object
 Recalling Firm City           object
 Recalling Firm State          object
 Recalling Firm Country        object
 Center Classification Date    object
 Reason for Recall             object
 Product Description           object
 Event ID                       int64
 Event Classification          object
 Product ID                     int64
 Center                        object
 Recall Details                object
 dtype: object)

In [4]:
# df.head()

In [5]:
#these are the unique product types the FDA regulates; it appears all product types have recalled products present in dataset
df['Product Type'].unique()

array(['Food/Cosmetics', 'Devices', 'Drugs', 'Veterinary', 'Biologics',
       'Tobacco'], dtype=object)

In [6]:
#checking for anomalies; veriable 'Distribution Pattern' has 1 null value
df.isnull().sum()

FEI Number                    0
Recalling Firm Name           0
Product Type                  0
Product Classification        0
Status                        0
Distribution Pattern          1
Recalling Firm City           0
Recalling Firm State          0
Recalling Firm Country        0
Center Classification Date    0
Reason for Recall             0
Product Description           0
Event ID                      0
Event Classification          0
Product ID                    0
Center                        0
Recall Details                0
dtype: int64

In [7]:
#calling record with null value
df[df['Distribution Pattern'].isna()]

,FEI Number,Recalling Firm Name,Product Type,Product Classification,Status,Distribution Pattern,Recalling Firm City,Recalling Firm State,Recalling Firm Country,Center Classification Date,Reason for Recall,Product Description,Event ID,Event Classification,Product ID,Center,Recall Details
2148,3012046682,FlexDex Inc.,Devices,Class II,Ongoing,NaN,Brighton,Michigan,United States,2022-01-27,The firm is reinforcing the IFU instructions: ...,"8mm FlexDex Needle Driver, Product Code FD-335 ND",89316,Class II,191253,CDRH,https://www.accessdata.fda.gov/scripts/ires/?P...


## EDA

In [8]:
#finding firms with the highest recalls
df.groupby('Recalling Firm Name')['Event ID'].count().sort_values(ascending=False)

Recalling Firm Name
Zimmer Biomet, Inc.                              875
Customed, Inc                                    682
OneBlood, Inc.                                   664
Garden-Fresh Foods, Inc.                         634
Aidapak Services, LLC                            538
                                                ... 
Joseph Shalhoub & Son, Inc.                        1
JosNoe Medical, Inc.                               1
Jones Natural Chews, Co., Inc.                     1
Joliet Blood Plasma LLC dba Plasma Care, Inc.      1
yoders country market                              1
Name: Event ID, Length: 8403, dtype: int64

In [9]:
#unique values in Event Classification column match FDA classes of severity of recalls
df['Event Classification'].unique()

array(['Class I', 'Class II', 'Class III'], dtype=object)

In [10]:
grouped_df = df.groupby(['Recalling Firm Name','Event Classification'])['Product Description'].count()
grouped_df

Recalling Firm Name       Event Classification
1 Epic, LLC               Class II                 4
1908 Brands               Class II                 6
2 Moms in the Raw, LLC    Class I                  1
21ST CENTURY SNACK FOODS  Class II                 1
21st Century Foods, Inc.  Class II                 1
                                                  ..
the Compounder            Class II                47
ulrich medical USA Inc    Class II                 3
weaver fundraising        Class II                 2
www.blankterrmall.com     Class I                  1
yoders country market     Class I                  1
Name: Product Description, Length: 10023, dtype: int64

First grouping results in 8,403 rows (firm name & the number of recalls they had). Second grouping results in 10,023 rows since I counted 'product decsription'. Difference is ~2,000. Need new filter to see what firms are repeating.

In [11]:
#filtering dataframe by class of Event Classification
class1 = df[df['Event Classification']=='Class I']
class2 = df[df['Event Classification']=='Class II']
class3 = df[df['Event Classification']=='Class III']
class1.shape, class2.shape, class3.shape

((16248, 17), (55269, 17), (6667, 17))

In [12]:
#filter option for encoding event classification by class into binary format
df['Class_I'] = np.where(df['Event Classification']=='Class I',1,0)
df['Class_II'] = np.where(df['Event Classification']=='Class II',1,0)
df['Class_III'] = np.where(df['Event Classification']=='Class III',1,0)

In [13]:
#deciding which features are categorical
df.nunique()

FEI Number                     8051
Recalling Firm Name            8403
Product Type                      6
Product Classification            3
Status                            3
Distribution Pattern          16937
Recalling Firm City            2950
Recalling Firm State             54
Recalling Firm Country           48
Center Classification Date     2752
Reason for Recall             25241
Product Description           67579
Event ID                      28501
Event Classification              3
Product ID                    78184
Center                            6
Recall Details                78184
Class_I                           2
Class_II                          2
Class_III                         2
dtype: int64

In [14]:
df.head()

,FEI Number,Recalling Firm Name,Product Type,Product Classification,Status,Distribution Pattern,Recalling Firm City,Recalling Firm State,Recalling Firm Country,Center Classification Date,Reason for Recall,Product Description,Event ID,Event Classification,Product ID,Center,Recall Details,Class_I,Class_II,Class_III
0,3010452309,ELITE CONFECTIONERY LTD,Food/Cosmetics,Class I,Ongoing,Distributed in the US nationwide,Nof Ha-Galil,-,Israel,2022-06-04,Potential contamination with Salmonella,Elite Hazelnut & Almond Milk Chocolate Bar Net...,90105,Class I,193404,CFSAN,https://www.accessdata.fda.gov/scripts/ires/?P...,1,0,0
1,3010452309,ELITE CONFECTIONERY LTD,Food/Cosmetics,Class I,Ongoing,Distributed in the US nationwide,Nof Ha-Galil,-,Israel,2022-06-04,Potential contamination with Salmonella,ELITE MILK CHOCOLATE BAR WITH STRAWBERRY CREAM...,90105,Class I,193406,CFSAN,https://www.accessdata.fda.gov/scripts/ires/?P...,1,0,0
2,3010452309,ELITE CONFECTIONERY LTD,Food/Cosmetics,Class I,Ongoing,Distributed in the US nationwide,Nof Ha-Galil,-,Israel,2022-06-04,Potential contamination with Salmonella,ELITE POPPING ROCKS MILK CHOCOLATE BAR UPC:815...,90105,Class I,193407,CFSAN,https://www.accessdata.fda.gov/scripts/ires/?P...,1,0,0
3,3010452309,ELITE CONFECTIONERY LTD,Food/Cosmetics,Class I,Ongoing,Distributed in the US nationwide,Nof Ha-Galil,-,Israel,2022-06-04,Potential contamination with Salmonella,ELITE WHITE CHOCOLATE BAR\tUPC:815871011471 & ...,90105,Class I,193408,CFSAN,https://www.accessdata.fda.gov/scripts/ires/?P...,1,0,0
4,3010452309,ELITE CONFECTIONERY LTD,Food/Cosmetics,Class I,Ongoing,Distributed in the US nationwide,Nof Ha-Galil,-,Israel,2022-06-04,Potential contamination with Salmonella,ELITE MILK CHOCOLATE WITH BERRY FLAVOR TRUFFLE...,90105,Class I,193409,CFSAN,https://www.accessdata.fda.gov/scripts/ires/?P...,1,0,0


In [15]:
#  encoding Event Classification
df['Event Classification'] = df['Event Classification'].replace({'Class I':1, 'Class II':2, 'Class III':3})

In [16]:
# encoding 'Product Type' and 'Recalling Firm Country'
for i in ['Product Type', 'Recalling Firm Country']:
    df[i] = encoder.fit_transform(df[[i]]).toarray()

In [17]:
# transformed dataframe with encoded 'Event Classification' and columns of interest filtered  out
new_df = df[['Recalling Firm Name', 'Product Type', 'Recalling Firm Country', 'Reason for Recall', 'Product Description', 'Event Classification']]
new_df.head()

,Recalling Firm Name,Product Type,Recalling Firm Country,Reason for Recall,Product Description,Event Classification
0,ELITE CONFECTIONERY LTD,0.0,0.0,Potential contamination with Salmonella,Elite Hazelnut & Almond Milk Chocolate Bar Net...,1
1,ELITE CONFECTIONERY LTD,0.0,0.0,Potential contamination with Salmonella,ELITE MILK CHOCOLATE BAR WITH STRAWBERRY CREAM...,1
2,ELITE CONFECTIONERY LTD,0.0,0.0,Potential contamination with Salmonella,ELITE POPPING ROCKS MILK CHOCOLATE BAR UPC:815...,1
3,ELITE CONFECTIONERY LTD,0.0,0.0,Potential contamination with Salmonella,ELITE WHITE CHOCOLATE BAR\tUPC:815871011471 & ...,1
4,ELITE CONFECTIONERY LTD,0.0,0.0,Potential contamination with Salmonella,ELITE MILK CHOCOLATE WITH BERRY FLAVOR TRUFFLE...,1


In [18]:
#table export
#new_df.to_csv('Exported_tables/ETL_data.csv')